In [7]:
# reload during dev
from importlib import reload
import PET_raw_data
PET_raw_data = reload(PET_raw_data)



dict_load = {
            #  '[Adt Income] adult.csv'      : {'describe_params' : {'missing_level' : {'data': {'?'}}}}
            # ,'[Census] adult.data'         : {'read_params'     : {'label_encoding' : 'Y'
            #                                                       ,'header_exist' : 'N'
            #                                                       ,'header'       : ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
            #                                                       }
            #                                  ,'describe_params' : {'missing_level' : {'data': {'?'}}}
            #                                  }
            # ,'[NHANES] B.csv'              : {'read_params'     : {'label_encoding' : 'Y'
            #                                                       ,'header_exist' : 'N'
            #                                                       ,'header'       : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
            #                                                       }
            #                                  }
            # ,
             '[Retail] Online Retail.xlsx' : {'read_method'     : 'pandas_xlsx'
                                             ,'read_params'     : {'label_encoding' : 'Y'
                                                                  ,'sheet_name' : 'Online Retail'
                                                                  ,'str_col'    : ['InvoiceNo','StockCode','InvoiceDate','CustomerID']
                                                                  }
                                             }
            ,'[UK-US PF] va_household_ver_1_9_instance_2.csv' : {'read_params' : {'label_encoding' : 'Y'
                                                                                 ,'str_col' : ['hid','rlid','admin1','admin2']}}
            ,'[UK-US PF] va_person_ver_1_9_instance_2.csv'    : {'read_params' : {'label_encoding' : 'Y'
                                                                                 ,'str_col' : ['hid','pid','person_number','sex']}}
            }
dict_out  = {'[Adt Income] adult.csv'      : "[Adt Income]"
            ,'[Census] adult.data'         : "[Census]"
            ,'[NHANES] B.csv'              : "[NHANES]"
            ,'[Retail] Online Retail.xlsx' : "[Retail]"
            ,'[UK-US PF] va_household_ver_1_9_instance_2.csv' : "[UK-US PF] Household"
            ,'[UK-US PF] va_person_ver_1_9_instance_2.csv'    : "[UK-US PF] Person"
            }



import time
from datetime import datetime
import pytz
from sdv.metadata import SingleTableMetadata
# 231103, Justyn: 如果是跑 VSCode 失敗的話，檢查能否 import torch，不行的話可能需要裝 Microsoft Visual C++ Redistributable：Microsoft 官網下載即可
from sdv.single_table import GaussianCopulaSynthesizer ,CTGANSynthesizer ,TVAESynthesizer ,CopulaGANSynthesizer
n_sample = 10
library_name = 'SDV'


__temp = {}
for filename ,file_params in dict_load.items():
    file_params_init = {'read_params'     : {'label_encoding' : 'Y'}
                       ,'describe'        : 'N'
                       ,'describe_params' : {'correlation'   : 'N'
                                            ,'collinearity'  : 'N'}
                       ,'report'          : 'N'
                       ,'report_params'   : {'print_report' : 'N'
                                            ,'save_report'  : 'Y'}
                       }
    file_params_init.update(file_params)
    print(f"Now is {filename}")
    __temp[filename] = PET_raw_data.load_data(f".\data\{filename}" ,file_params_init)



    time_start = time.time()
    __data         = __temp[filename].data.copy()
    __data_row_num = __data.shape[0]
    metadata       = SingleTableMetadata()
    metadata.detect_from_dataframe(__data)
    ##
    __data_oricast   = __temp[filename].data_oricast.copy()
    metadata_oricast = SingleTableMetadata()
    metadata_oricast.detect_from_dataframe(__data)
    ##
    __data_label_encoding   = __temp[filename].data_label_encoding.copy()
    metadata_label_encoding = SingleTableMetadata()
    metadata_label_encoding.detect_from_dataframe(__data_label_encoding)
    print(f"讀 3*metafile 時間 {round(time.time()-time_start ,4)} 秒")

    for method_name in ['GaussianCoupula'
                       ,'CTGAN'
                       ,'TVAE'
                       ,'CoupulaGAN']:

        if filename == '[Retail] Online Retail.xlsx' and method_name == 'GaussianCoupula':
            continue
        elif filename == '[Retail] Online Retail.xlsx' and method_name == 'CTGAN':
            continue

        exectime = datetime.now().astimezone(pytz.timezone('Asia/Taipei'))
        exec_out = f"{dict_out[filename]}_[{library_name}]_[{method_name}]_{exectime.strftime('%Y%m%d_%H%M%S')}_{exectime.tzinfo.zone.replace('/' ,'_')}"

        print(f"現在套件是{method_name}")
        time_start = time.time()
        if   method_name == 'GaussianCoupula':
            synthesizer = GaussianCopulaSynthesizer(metadata)
            synthesizer.fit(__data)
        elif method_name == 'CTGAN':
            synthesizer = CTGANSynthesizer(metadata_oricast)
            synthesizer.fit(__data_oricast)
        elif method_name == 'TVAE':
            synthesizer = TVAESynthesizer(metadata_label_encoding)
            synthesizer.fit(__data_label_encoding)
        elif method_name == 'CoupulaGAN':
            synthesizer = CopulaGANSynthesizer(metadata_label_encoding)
            synthesizer.fit(__data_label_encoding)
        else:
            continue
        print(f"訓練時間 {round(time.time()-time_start ,4)} 秒")
        synthesizer.save(f".\model_dpsd\{exec_out}.pkl")


        time_start = time.time()
        for i in range(n_sample):
            synthesizer.reset_sampling()
            synthetic_data = synthesizer.sample(num_rows   = 10000
                                            ,batch_size = 1000
                                            )
            synthetic_data.to_csv(f".\data_dpsd\{exec_out}_10000.csv")
        print(f"合成 {n_sample} 次 10K 筆時間 {round(time.time()-time_start ,4)} 秒")

        time_start = time.time()
        synthetic_data = synthesizer.sample(num_rows   = __data_row_num
                                        ,batch_size = 1000
                                        )
        synthetic_data.to_csv(f".\data_dpsd\{exec_out}_{__data_row_num}.csv")
        print(f"合成一次同筆數時間 {round(time.time()-time_start ,4)} 秒")



        print(f"Done: {exec_out}")

Now is [Retail] Online Retail.xlsx


讀 3*metafile 時間 0.6815 秒
現在套件是TVAE


C:\Users\justyn.chen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'UnitPrice'. Data will not be rounded.
  warnings.warn(


訓練時間 22726.3077 秒


Sampling rows: 100%|██████████| 10000/10000 [00:00<00:00, 13793.16it/s]


...

Sampling rows: : 542000it [15:17, 590.89it/s]                           


合成一次同筆數時間 918.997 秒
Done: [Retail]_[SDV]_[CoupulaGAN]_20231106_231040_Asia_Taipei
Now is [UK-US PF] va_household_ver_1_9_instance_2.csv
讀 3*metafile 時間 5.81 秒
現在套件是GaussianCoupula
訓練時間 3374.5124 秒


Sampling rows: 100%|██████████| 10000/10000 [05:00<00:00, 33.32it/s] 


合成 10 次 10K 筆時間 3035.5302 秒


Sampling rows: : 3095000it [1:34:05, 475.94it/s]                           